# Exercise 2
Please classify this flower using KNN (with all libs available as you like).

### Iris Import
Zunächst muss das Iris-Dataset importiert werden. Hierbei wird der Datensatz in iris_data und iris_labels aufgeteilt. iris_data umfasst hierbei die Daten der gemessenen Sepal und Petal Längen und Breiten, während iris_labels die Arten der iris-Pflanze (Setosa, Versicolor, Virginica) passend zum Datensatz abbildet.

In [1]:
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()
iris_data = iris.data
iris_labels = iris.target

iris_data[0]

array([5.1, 3.5, 1.4, 0.2])

### Creating Learnset
Zunächst wird ein Learnset erstellt, dessen Basis eine durch Zufall gewählte Zahl ist, um den Datensatz zufällig aufteilen zu können. Zusätzlich wird festgelegt, dass es genau 15 Datensätze für das Learnset geben soll. Die so ausgewählten Daten werden in eigenen Variablen, learnset_data und learnset_labels gespeichert. 

Anschließend wird die durch die Aufgabe gegebene Iris (4.8,2.5,5.3,2.4) genutzt, um die Daten für das Testset einzusetzen. Im Rahmen dieser Ausführungen soll erkannt werden zu welcher Iris-Gruppe diese Iris gehört.

In [2]:
np.random.seed(13)
indices = np.random.permutation(len(iris_data))
n_training_samples = 15

learnset_data = iris_data[indices[:-n_training_samples]]
learnset_labels = iris_labels[indices[:-n_training_samples]]

testset_data = [4.8, 2.5, 5.3, 2.4]

testset_data

[4.8, 2.5, 5.3, 2.4]

### Euklidische Abstandsfunktion und Normalisierung
Anschließend wird eine Funktion für den euklidischen Abstand und Normalisierung erstellt. Diese wird in einem Schritt vorgenommen. Zwei Punkte werden voneinander subtrahiert und das Ergebnis dieses Schrittes wird durch die numpy-Funktion linalg.norm() normalisiert.

In [3]:
def distance(instance1, instance2):
    instance1 = np.array(instance1) 
    instance2 = np.array(instance2)    
    return np.linalg.norm(instance1 - instance2)

print(distance(learnset_data[13], learnset_data[42]))

2.696293752542553


### Nachbarn finden
In diesem Schritt wird eine Funktion erstellt, die k Nachbarn eines Punktes finden kann. Hierfür wird die Distanz der Punkte in einer For-Schleife errechnet und anschließend sortiert. Nach der Sortierung werden die k dichtesten Nachbarn zurückgegeben. 

Eingesetzt wird dies in dem Iris-Beispiel für die k=3 dichtesten Nachbarn, sodass der Aufruf der so erstellten Funktion genau drei Ergebnisse zurückliefert. 

In [4]:
def get_neighbors(training_set, labels, test_instance, k, distance=distance):
    distances = []
    for index in range(len(training_set)):
        dist = distance(test_instance, training_set[index])
        distances.append((training_set[index], dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors

In [5]:
neighbors = get_neighbors(learnset_data, learnset_labels, testset_data, 3, distance=distance)

print(neighbors[0])
print(neighbors[1])
print(neighbors[2])

(array([5.6, 2.8, 4.9, 2. ]), 1.0246950765959595, 2)
(array([5.7, 2.5, 5. , 2. ]), 1.0295630140987002, 2)
(array([5.8, 2.8, 5.1, 2.4]), 1.0630145812734648, 2)


### Target Class ermitteln
Anhand der get_neighbors-Funktion kann bereits erkannt werden, dass durch k=3 Ergebnisse drei Mal die 2 als Target Klasse vor kommt. Damit kann schon jetzt abgelesen werden, dass die 2 als Ergebnis aus dem Test hervor geht. Zusätzlich kann jedoch eine weitere Funktion erstellt werden, welche es ermöglicht eben diese ausgewählten Target Klassen zu erkennen und diese als ein Vote-Result darzustellen. 
Hierfür wurde eine vote()-Funktion erstellt, die das Ergebnis der Nachbarn nutzt, um pro Durchlauf der For-Schleife einen Counter hochzuzählen, der zählt, wie oft welche Target Klasse in den Nachbarn erkannt wird. Anschließend wird die am häufigsten genannte Klasse als Vote-Ergebnis wiedergegeben. Auch durch diese Funktion wird schließlich die 2 als Target Class ermittelt. Die 2 ist in diesem Fall das Label für die Iris Virginica.

In [6]:
from collections import Counter
def vote(neighbors):
    class_counter = Counter()
    for neighbor in neighbors:
        class_counter[neighbor[2]] += 1
    return class_counter.most_common(1)[0][0]

In [7]:
neighbors = get_neighbors(learnset_data, learnset_labels, testset_data, 3, distance=distance)

print("result of vote: ", vote(neighbors),", data: ", testset_data)

result of vote:  2 , data:  [4.8, 2.5, 5.3, 2.4]
